# New Section

In [1]:

import sys
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import sklearn.model_selection





from tqdm import tqdm
import random 
import datetime

import joblib
import pickle
import tempfile


import random 
import datetime


#'''
#SAGEMAKER:
#!conda install -c conda-forge lightgbm

import boto3
import sagemaker
from sagemaker import get_execution_role


# sagemaker session, role
session = sagemaker.Session()
role = sagemaker.get_execution_role()


# S3 bucket name
bucket = 'sagemaker-us-west-2-479862333671'
subfolder = ''
conn = boto3.client('s3')
contents = conn.list_objects(Bucket=bucket, Prefix=subfolder)['Contents']


**Load the data**

In [2]:
train_df = pd.read_csv('s3://{}/{}'.format(bucket, 'data/train_df.csv'))

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,13,14,15) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train_df

,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,1437,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
1,1438,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
2,1439,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
3,1440,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
4,1441,1,0,0,1,0,2011-01-29,11101,Saturday,1,1,2011,NaN,NaN,NaN,NaN,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53967295,1432,0,9,2,1770,1,2015-12-03,11544,Thursday,6,12,2015,NaN,NaN,NaN,NaN,1,1,1
53967296,1433,0,9,2,1770,0,2015-12-03,11544,Thursday,6,12,2015,NaN,NaN,NaN,NaN,1,1,1
53967297,1434,0,9,2,1770,1,2015-12-03,11544,Thursday,6,12,2015,NaN,NaN,NaN,NaN,1,1,1
53967298,1435,0,9,2,1770,0,2015-12-03,11544,Thursday,6,12,2015,NaN,NaN,NaN,NaN,1,1,1


In [10]:
list(train_df)

['item_id',
 'cat_id',
 'store_id',
 'state_id',
 'd',
 'sales',
 'date',
 'wm_yr_wk',
 'weekday',
 'wday',
 'month',
 'year',
 'event_name_1',
 'event_type_1',
 'event_name_2',
 'event_type_2',
 'snap_CA',
 'snap_TX',
 'snap_WI']

In [4]:
val_df = pd.read_csv('s3://{}/{}'.format(bucket, 'data/val_df.csv'))

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
val_df

,item_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,1437,1,0,0,1771,0,2015-12-04,11544,Friday,7,12,2015,NaN,NaN,NaN,NaN,1,0,0
1,1438,1,0,0,1771,0,2015-12-04,11544,Friday,7,12,2015,NaN,NaN,NaN,NaN,1,0,0
2,1439,1,0,0,1771,0,2015-12-04,11544,Friday,7,12,2015,NaN,NaN,NaN,NaN,1,0,0
3,1440,1,0,0,1771,3,2015-12-04,11544,Friday,7,12,2015,NaN,NaN,NaN,NaN,1,0,0
4,1441,1,0,0,1771,1,2015-12-04,11544,Friday,7,12,2015,NaN,NaN,NaN,NaN,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853715,1432,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
853716,1433,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
853717,1434,0,9,2,1798,1,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0
853718,1435,0,9,2,1798,0,2015-12-31,11548,Thursday,6,12,2015,NaN,NaN,NaN,NaN,0,0,0


In [6]:
X_df = pd.read_csv('s3://{}/{}'.format(bucket, 'data/X_df.csv'))

In [7]:
X_df

,0,1,2,3,4,5,6
0,1437,1,0,0,1,1,2011
1,1438,1,0,0,1,1,2011
2,1439,1,0,0,1,1,2011
3,1440,1,0,0,1,1,2011
4,1441,1,0,0,1,1,2011
...,...,...,...,...,...,...,...
53967295,1432,0,9,2,6,12,2015
53967296,1433,0,9,2,6,12,2015
53967297,1434,0,9,2,6,12,2015
53967298,1435,0,9,2,6,12,2015


In [8]:
y_df = pd.read_csv('s3://{}/{}'.format(bucket, 'data/y_df.csv'))

In [9]:
y_df

,0
0,0
1,0
2,0
3,0
4,0
...,...
53967295,1
53967296,0
53967297,1
53967298,0
